# Creating Features Quiz
Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [37]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, \
    IDF, StringIndexer
from pyspark.sql.functions import udf, concat
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import MinMaxScaler, Normalizer, StandardScaler, \
    VectorAssembler
import re

# TODOS: 
# 1) import any other libraries you might need
# 2) run the cells below to read dataset and build body length feature
# 3) write code to answer the quiz questions 

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [3]:
stack_overflow_data = 'Train_onetag_small.json'

In [23]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

In [9]:
df.columns

['Body', 'Id', 'Tags', 'Title', 'oneTag', 'words', 'BodyLength']

### Build Body Length Feature

In [25]:
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)

In [26]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("BodyLength", body_length(df.words))

In [27]:
df = df.select(concat('Title','Body').alias('full_text'), 'Id', 'Tags','oneTag', 'words', 'BodyLength')

In [ ]:
df.head()

# Question 1
Select the question with Id = 1112. How many words does its body contain (check the BodyLength column)?

In [8]:
df.select('BodyLength').where(df.Id==1112).show()
# TODO: write your code to answer question 1

+----------+
|BodyLength|
+----------+
|        63|
+----------+



# Question 2
Create a new column that concatenates the question title and body. Apply the same functions we used before to compute the number of words in this combined column. What's the value in this new column for Id = 5123?

In [28]:
regexTokenizer1 = RegexTokenizer(inputCol="full_text", outputCol="all_words", pattern="\\W")
df = regexTokenizer1.transform(df)

In [29]:
text_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("FullTextLength", text_length(df.all_words))

In [13]:
df.select('FullTextLength').where(df.Id==5123).show()
# TODO: write your code to answer question 2

+--------------+
|FullTextLength|
+--------------+
|           135|
+--------------+



# Create a Vector
Create a vector from the combined Title + Body length column. In the next few questions, you'll try different normalizer/scaler methods on this new column.

In [39]:
assembler = VectorAssembler(inputCols=["FullTextLength"], outputCol="vector_text")
df2 = assembler.transform(df)
# TODO: write your code to create this vector

# Question 3
Using the Normalizer method what's the normalized value for question Id = 512?

In [41]:
norm = Normalizer(inputCol="vector_text", outputCol="norm_count")
transformed = norm.transform(df2)

# TODO: write your code to answer question 3

In [46]:
transformed.select('norm_count').where(transformed.Id==512).collect()

[Row(norm_count=DenseVector([1.0]))]

# Question 4
Using the StandardScaler method (scaling both the mean and the standard deviation) what's the normalized value for question Id = 512?

In [49]:
stdnorm = StandardScaler(inputCol="vector_text", outputCol="norm_count", withStd=True, withMean = True)
fitted = stdnorm.fit(df2)
std_norm = fitted.transform(df2)
# TODO: write your code to answer question 4

In [50]:
std_norm.select('norm_count').where(std_norm.Id==512).collect()

[Row(norm_count=DenseVector([-0.6417]))]

# Question 5
Using the MinMAxScaler method what's the normalized value for question Id = 512?

In [54]:
minmax = MinMaxScaler(inputCol="vector_text", outputCol="norm_count")
fitted = minmax.fit(df2)
minmax_norm = fitted.transform(df2)
# TODO: write your code to answer question 5

In [55]:
minmax_norm.select('norm_count').where(std_norm.Id==512).collect()

[Row(norm_count=DenseVector([0.0062]))]